In [1]:
import csv
import os
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [26]:
from app.flipside.models import Flipside
from app.constants import (oracle_feeds_lookup, 
    oracle_feeds, 
    phase_windows, 
    liquidators, 
    attackers, 
    targets, 
    filenames_filtered, 
    filenames)

In [5]:
phase_windows = [
# [14053849, 14424164], # Michael Initial
    [14424164, 15962578, 'Phase: Michael Ramp Up', 'blue'], # Michael Ramp Up
    [15962578, 16025860, 'Avi Setup', 'grey'], # Avi Setup
    [16025860, 16025896, 'Avi Liquidation 1', 'red'], # Avi Liquidation 1
    [16025896,16027007, 'Between Liquidations', ""],  # Jockeying
    [16027007, 16027236, 'Avi Liquidation 2', 'red'], # Avi Liquidation 2
] 

## Variables

In [127]:
start = 15962578 -10
end = phase_windows[4][1]
title = f"{phase_windows[1][2]} - {phase_windows[2][2]}"

In [128]:
address = '0x57e04786e231af3343562c062e0d058f25dace9e'
end = 16027236

## Utility

In [119]:

def apply_window(df, window_start=start, window_end=end):
    return df.loc[
        (df["BLOCK_NUMBER"] > window_start) & 
        (df["BLOCK_NUMBER"] < window_end)
    ]

def chart_it(title, df, x, y, color, facet_col="SYMBOL"):
    fig = px.scatter(df, 
                     x=x,
                     y=y,
                     color=color,
                    title=title,
                    facet_col=facet_col,
)
    fig.update_layout(autotypenumbers='convert types')
    fig.show()

def fig_append_phase_labels(fig, start = None, end=None):
    for window in phase_windows:
        if window[0]>= start and window[1]<= end and window[3] != "":
            fig.add_vrect(x0=window[0], x1=window[1], 
                        annotation_text=window[2], annotation_position="top left",
                        fillcolor=window[3], opacity=0.25, line_width=0)
    return fig

In [120]:
def process(fs, address, start, end):
    sums = {}
    # Deposits
    df = fs.dataframes['aave_deposits']
    
    deposits = df
    deposits = deposits[deposits['DEPOSITOR_ADDRESS']== address]
    deposits = deposits[deposits['SYMBOL']== 'CRV']
    deposits.keys()
    df_filtered = apply_window(deposits, 14053849, end)
    out = df_filtered.groupby(["DEPOSITOR_ADDRESS", "SYMBOL"]).ISSUED_TOKENS.sum().reset_index()
    try:
        # print(float(out.ISSUED_TOKENS))
        sums['deposit'] = float(out.ISSUED_TOKENS)
    except:
        # print(None)
        sums['deposit'] = float(0)

    # Withdraws
    df = fs.dataframes['aave_withdraws']

    deposits = df
    deposits = deposits[deposits['DEPOSITOR_ADDRESS']== address]
    deposits = deposits[deposits['SYMBOL']== 'CRV']
    deposits.keys()
    df_filtered = apply_window(deposits, 14053849, end)
    out = df_filtered.groupby(["DEPOSITOR_ADDRESS", "SYMBOL"]).WITHDRAWN_TOKENS.sum().reset_index()
    try: 
        # print(float(out.WITHDRAWN_TOKENS))
        sums['withdrawn'] = float(out.WITHDRAWN_TOKENS)
    except:
        # print(None)
        sums['withdrawn'] = float(0)

    # Borrows
    df = fs.dataframes['aave_borrows']

    deposits = df
    deposits = deposits[deposits['BORROWER_ADDRESS']== address]
    deposits = deposits[deposits['SYMBOL']== 'CRV']
    deposits.keys()
    df_filtered = apply_window(deposits, 14053849, end)
    out = df_filtered.groupby(["BORROWER_ADDRESS", "SYMBOL"]).BORROWED_TOKENS.sum().reset_index()
    try:
        # print(float(out.BORROWED_TOKENS))
        sums['borrowed'] = float(out.BORROWED_TOKENS)

    except:
        # print(None)
        sums['borrowed'] = float(0)

    # Repayed
    df = fs.dataframes['aave_repayments']

    deposits = df
    deposits = deposits[deposits['BORROWER']== address]
    deposits = deposits[deposits['SYMBOL']== 'CRV']
    deposits.keys()
    df_filtered = apply_window(deposits, 14053849, end)
    out = df_filtered.groupby(["BORROWER", "SYMBOL"]).REPAYED_TOKENS.sum().reset_index()
    try:
        # print(float(out.REPAYED_TOKENS))
        sums['repayed'] = float(out.REPAYED_TOKENS)
    except:
        # print(None)
        sums['repayed'] = float(0)


    # Liquidatied Debt
    df = fs.dataframes['aave_liquidations']

    deposits = df
    deposits = deposits[deposits['BORROWER']== address]
    deposits = deposits[deposits['DEBT_TOKEN_SYMBOL']== 'CRV']
    deposits.keys()
    df_filtered = apply_window(deposits, 14053849, end)
    out = df_filtered.groupby(["BORROWER", "DEBT_TOKEN_SYMBOL"]).DEBT_TO_COVER_AMOUNT.sum().reset_index()
    try:
        # print(float(out.DEBT_TO_COVER_AMOUNT))
        sums['liquidated_debt'] = float(out.DEBT_TO_COVER_AMOUNT)
    except:
        # print(None)
        sums['liquidated_debt'] = float(0)
    
    return sums

# Run

In [121]:
fs = Flipside(filenames_filtered)

Loading...
v2_deposits_actors
v2_borrows_actors
v2_liquidations_actors
v2_repayments_actors
v2_withdraws_actors
crv_transfers-download-2022-12-12-01-35-pm-results
stkaave_transfers-download-2022-12-12-01-35-pm-results
stkabpt_transfers-download-2022-12-12-01-35-pm-results
chainlink_all_oracle_results
aave_liquidations
['BLOCK_NUMBER', 'LIQUIDATED_AMOUNT', 'LIQUIDATED_AMOUNT_USD', 'COLLATERAL_TOKEN_PRICE', 'DEBT_TO_COVER_AMOUNT', 'DEBT_TO_COVER_AMOUNT_USD', 'DEBT_TOKEN_PRICE']
BLOCK_NUMBER
LIQUIDATED_AMOUNT
LIQUIDATED_AMOUNT_USD
COLLATERAL_TOKEN_PRICE
DEBT_TO_COVER_AMOUNT
DEBT_TO_COVER_AMOUNT_USD
DEBT_TOKEN_PRICE
aave_borrows
['BLOCK_NUMBER', 'BORROWED_TOKENS', 'BORROWED_USD', 'TOKEN_PRICE']
BLOCK_NUMBER
BORROWED_TOKENS
BORROWED_USD
TOKEN_PRICE
aave_repayments
['BLOCK_NUMBER', 'REPAYED_TOKENS', 'REPAYED_USD', 'TOKEN_PRICE']
BLOCK_NUMBER
REPAYED_TOKENS
REPAYED_USD
TOKEN_PRICE
aave_withdraws
['BLOCK_NUMBER', 'WITHDRAWN_USD', 'WITHDRAWN_TOKENS', 'TOKEN_PRICE']
BLOCK_NUMBER
WITHDRAWN_USD
WI

In [129]:
process(fs, address, start, 16025860)

{'deposit': 0.0,
 'withdrawn': 0.0,
 'borrowed': 92000000.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0}

In [123]:
michael = {
14710000:
 {'deposit': 8400000.0,
 'withdrawn': 0.0,
 'borrowed': 0.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0
 },

15100000:
 {'deposit': 152518808.9776234,
 'withdrawn': 23900000.0,
 'borrowed': 0.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0
 },

15900000:
 {'deposit': 155518808.9776234,
 'withdrawn': 23900000.0,
 'borrowed': 0.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0
 },

15962578:
 {'deposit': 215963568.4168177,
 'withdrawn': 42900000.0,
 'borrowed': 0.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0
 },
 16025860:
 {'deposit': 361963568.41681767,
 'withdrawn': 228900000.0,
 'borrowed': 0.0,
 'repayed': 0.0,
 'liquidated_debt': 0.0}
}



503621289.746133

503621289.746133

In [109]:
data = []
for r in michael:
    data.append(michael[r]['deposit'] -michael[r]['withdrawn'])
print(data)

print(f"1 / 4 \n\t{data[0]/data[3]} | {data[3] - data[0]}")
print(f"2 / 4 \n\t{data[1]/data[3]} | {data[3] - data[1]}")
print(f"3 / 4 \n\t{data[2]/data[3]} | {data[3] - data[2]}")


comparator = 2

print(f"1 / 4 \n\t{data[0]/data[comparator]} | {data[comparator] - data[0]}")
print(f"2 / 4 \n\t{data[1]/data[comparator]} | {data[comparator] - data[1]}")
print(f"3 / 4 \n\t{data[3]/data[comparator]} | {data[comparator] - data[3]}")

[8400000.0, 128618808.9776234, 131618808.9776234, 173063568.4168177, 133063568.41681767]
1 / 4 
	0.04853707846684917 | 164663568.4168177
2 / 4 
	0.7431882409118561 | 44444759.43919429
3 / 4 
	0.7605229117928737 | 41444759.43919429
1 / 4 
	0.0638206656423102 | 123218808.9776234
2 / 4 
	0.9772069051277463 | 3000000.0
3 / 4 
	1.3148847779517618 | -41444759.43919429


In [124]:
print(f"3 / 4 \n\t{data[3]/data[4]} | {data[4] - data[3]}")

3 / 4 
	1.3006082015980605 | -40000000.00000003


In [125]:
print(92000000 /173063568.4168177) # % borrowed 
print(92000000 / 133063568.41681767) # % borrowed by 1st liquidation
0.335593 + 0.5315965736845385 # % utilized

0.5315965736845385
0.6913988636755385


0.8671895736845385

In [117]:
# df = fs.dataframes['aave_deposits']
# print(len(df))
# df_filtered = apply_window(df, start, 14740122)
# crv_deposits = df_filtered[df_filtered['SYMBOL']== 'CRV']
# crv_deposits[['BLOCK_NUMBER', 'ISSUED_TOKENS']].tail(10)